# Preprocesamiento de datos en MNE-Python

`
Autores:
Brigitte Aguilar, Sofía Poux, Elizabeth Young
`

Modificado de *PracticalMEEG2022: MNE-python hands-on tutorial*. Por Britta Westner. 

## Setup

Comenzaremos por cargar los paquetes que necesitaremos. Dentro de ellos se incluyen `matplotlib` para graficación, `os` para el manejo de directorios de archivos, `numpy` para operaciones numéricas y, por supuesto, `mne`.
También usaremos matplotlib magic para graficar las figuras en la misma notebook (inline). 

In [1]:
%matplotlib inline 
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mne
plt.style.use('ggplot')

Comprobemos nuevamente nuestra versión MNE-Python. Esto nos debería devolver 1.5.1 o una versión previa

In [2]:
mne.__version__

'1.5.1'

Configuramos el nivel de detalle de los mensajes de salida para que sea menos detallada:

In [3]:
mne.set_log_level('warning')

### Ayuda!

Recordá, si necesitas ayuda podés pedirla utilizando el signo de interrogación. 
Veamos cómo podemos obtener la documentación de una función. Por ejemplo, de la función `pick_types`.

In [ ]:
mne.pick_types?

## Estableciendo la ruta de los datos

En primer lugar, debes descargar la carpeta del dataset `Datos`. Luego, dejaremos a Python saber dónde encontrar esta carpeta en nuestro disco. Para ello, deberás modificar la ruta de abajo para que se ajuste a la estructura de la ruta de tu computadora!.
Podés imprimir la ruta completa para verificar que el directorio sea correcto. 

In [4]:
# Modificá la siguiente ruta según dónde se encuentre la carpeta Datos en tu disco
data_path = os.path.expanduser("~/Documents/MNE-projects/mne_tutorial_GardenOroVerde/Datos/")

raw_fname = os.path.join(data_path,
    'sub-01/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

In [ ]:
print(raw_fname)

Utilizá `bash` para verificar que la ruta se encuentra allí. Si esto te da un error, es problable que hayas cometido un error de tipeo en la ruta!

In [ ]:
ls $raw_fname

## Acceso y lectura de los datos crudos

In [ ]:
mne.io.read_raw_fif?

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=False)
print(raw)

In [ ]:
raw

Para más información sobre cómo importar datos en MNE, visitá:
- para MEG: https://mne.tools/stable/auto_tutorials/io/plot_10_reading_meg_data.html
- para EEG: https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html

## Entendiendo el archivo de los datos

Veamos la información de las mediciones. Allí encontrarás detalles sobre:
   - frecuencia de muestreo
   - parámetros de filtrado
   - tipos de canales disponibles 
   - canales defectuosos
   - etc.

In [ ]:
print(raw.info)

<div class="alert alert-success">
    <b>Ejercicio</b>:
     <ul>
    <li>¿Cuántos canales tenés para cada uno de los tipos de sensores?</li>
    <li>¿Cuál es la frecuencia de muestreo?</li>
    <li>¿Los datos fueron filtrados?</li>
    <li>¿Cuál es la frecuencia del ruido de línea?</li>
    <li>¿Existe algún canal defectuoso?</li>
    </ul>
</div>

## Una mirada más cercana al diccionario info

raw.info no es más que un diccionario:

In [ ]:
isinstance(raw.info, dict)

Por lo tanto, podemos acceder a sus elementos de esta forma:

In [ ]:
raw.info['sfreq']  # Frecuencia de muestreo

In [ ]:
raw.info['bads']  # lista de canales defectuosos

In [ ]:
raw.info['line_freq']  # frecuencia del ruido de línea

## Una mirada más cercana a los canales
Ahora veamos qué canales están presentes. Esto lo podemos ver a través del atributo `raw.ch_names`.

In [ ]:
type(raw.ch_names)

In [ ]:
raw.ch_names[:10]  # esto imprime los primeros 10 canales

Podés indexarlos como una lista.

In [ ]:
raw.ch_names[42]

También podemos consultar el tipo de canal de un canal específico:

In [ ]:
channel_type = mne.io.pick.channel_type(raw.info, 75)
print('Canal #75 es de tipo:', channel_type)  # para imprimir de manera ordenada

channel_type = mne.io.pick.channel_type(raw.info, 320)
print('Canal #320 es de tipo:', channel_type)

La información también contiene todos los detalles sobre los sensores (tipo, ubicaciones, marco coordenado, etc.) en `chs`:

In [ ]:
len(raw.info['chs'])

In [ ]:
type(raw.info['chs'])

In [ ]:
raw.info['chs'][0]  # chequeamos el primer canal

In [ ]:
raw.info['chs'][330]

Ahora que sabemos que hay canales de EEG y MEG en los datos, podemos graficar ambos por separado. Las posiciones de los canales están disponibles en el atributo info del objeto raw, por lo tanto, podemos graficar las ubicaciones de los mismos directamente desde el objeto raw utilizando `plot_sensors()`.

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='grad');

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

## Configurando los tipos de canales y re-referenciando

Algunos canales están definidos erróneamente como EEG en el archivo. 
Dos de ellos son de EOG (EEG061 y EEG062) y EEG063 es una canal de ECG. EEG064 estaba registrando pero no estaba conectado a nada así que lo haremos `'misc'` en su lugar. 
Ahora configuraremos los tipos de canales para aquellos canales clasificados incorrectamente. Esto será útil para el rechazo automático de artefactos.

In [ ]:
raw.set_channel_types?

In [ ]:
raw.set_channel_types({'EEG061': 'eog',  #  EOG no EEG
                       'EEG062': 'eog',  #  EOG no EEG
                       'EEG063': 'ecg',  #  ECG no EEG
                       'EEG064': 'misc'})  # EEG064 no conectado

# Renombramos los canales de EOG y ECG:
raw.rename_channels({'EEG061': 'EOG061',
                     'EEG062': 'EOG062',
                     'EEG063': 'ECG063'})

In [ ]:
raw.info

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

Una vez que hayamos arreglado los canales, podemos re-referenciar los canales de EEG al promedio de los mismos. 

In [ ]:
# Para configurar la referencia debemos cargar los datos en la memoria:
raw.load_data()
print(raw.info['custom_ref_applied'])  # veamos si existe una referencia

In [ ]:
# ahora re-referenciamos
raw.set_eeg_reference(ref_channels='average', projection=False)
print(raw.info['projs'])  # no agregado como proyección
print(raw.info['custom_ref_applied'])

## Accediendo a los datos

Para acceder a los datos sólo debes utilizar `[]` tal como se accede a cualquier elemento de una lista, diccionario, etc. Vemos que `raw[]` devuelve dos cosas: los datos y un arreglo de instantes de tiempo.

In [ ]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # acceder a todos los canales para los primeros 10 instantes de tiempo
print(data.shape)
print(times.shape)

In [ ]:
times  # siempre comienzan en 0 por convención

Vimos que `raw[]` devuelve ambas cosas, los datos y los instantes de tiempo.

## Remuestreando los datos

Ahora cambiaremos la frecuencia de muestreo de los datos para acelerar los cálculos..

In [ ]:
raw.load_data()  # cargar datos en memoria
raw.resample(300)

Y eliminemos los canales innecesarios: algunos canales de estímulo vacíos, canales misc y canales HPI.

In [ ]:
raw.drop_channels?

In [37]:
to_drop = ['STI201', 'STI301', 'MISC201', 'MISC202', 'MISC203',
           'MISC204', 'MISC205', 'MISC206', 'MISC301', 'MISC302',
           'MISC303', 'MISC304', 'MISC305', 'MISC306', 'CHPI001',
           'CHPI002', 'CHPI003', 'CHPI004', 'CHPI005', 'CHPI006',
           'CHPI007', 'CHPI008', 'CHPI009']

In [ ]:
raw.drop_channels(to_drop)

## Filtrando los datos y graficando los datos crudos

Filtraremos los datos entre 0 y 40 Hz utilizando un filtro de respuesta finita al impulso (FIR) de fase lineal.

<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
      <li>¿Qué parámetros debemos configurar para realizar dicho filtrado, basándonos en la documentación del método `filter`?</li>
    </ul>
</div>


In [ ]:
raw.filter?

Para ver qué efecto tiene el filtrado en nuestros datos, primero grafiquemoslos rápidamente. Para una funcionalidad completa, le pedimos a matplotlib que muestre el gráfico en una ventana separada.

In [ ]:
%matplotlib qt
raw.plot()

In [ ]:
raw.filter(0, 40)

Ahora que filtramos nuestros datos, veámoslos nuevamente. ¿Podés ver la diferencia?

In [ ]:
raw.plot()

<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
      <li> ¿Qué señales cambiaron más debido al fitrado: EEG o MEG?</li>
      <li> ¿A qué podría deberse?</li>
      <li> ¿Observaste algún canal defectuoso?</li>
      <li> ¿Cuáles son las características más relevantes que podés observar en los datos?</li>
       </ul>
</div>

Para obtener más información sobre la visualización de datos crudos, consultá aquí: 
https://mne.tools/0.16/auto_tutorials/plot_visualize_raw.html


## Una mirada a la estructura de eventos de los datos

Los datos tienen diferentes eventos que marcan qué estímulo se presentó a los participantes. La estructura de evento/trigger es la siguiente:
- 5, 6, 7: rostros famosos
- 13, 14, 15: rostros no familiares
- 17, 18, 19: rostros mezclados

Primero veamos qué eventos hay:

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', verbose=True)

In [ ]:
events

In [ ]:
mne.count_events(events, ids=[5])

<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
    <li>¿De qué tipo es la variable events?</li>
    <li>¿Cuál es el significado de las tres columnas de events?</li>
    <li>¿Cuántos eventos con código 5 hay?
    </ul>
</div>

 

Hubo un retraso temporal de 34,5 ms en la presentación del estímulo. Por lo cual necesitamos corregir los eventos.

In [46]:
delay = int(round(0.0345 * raw.info['sfreq']))
events[:, 0] = events[:, 0] + delay

Visualicemos el paradigma:

In [47]:
events = events[events[:, 2] < 20]  # toma sólo los eventos con código menor a 20

In [48]:
fig = mne.viz.plot_events(events, raw.info['sfreq']);

Para etiquetar los eventos y condiciones utilizamos un diccionario de Python con claves que contienen "/" para agrupar subcondiciones.

In [49]:
event_id = {
    'face/famous/first': 5,
    'face/famous/immediate': 6,
    'face/famous/long': 7,
    'face/unfamiliar/first': 13,
    'face/unfamiliar/immediate': 14,
    'face/unfamiliar/long': 15,
    'scrambled/first': 17,
    'scrambled/immediate': 18,
    'scrambled/long': 19,
}

In [50]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          event_id=event_id);

Ahora podemos volver a ver nuestros datos crudos con las etiquetas de eventos modificadas:

In [51]:
raw.plot(event_id=event_id, events=events);

## Creación de épocas y rechazo de artefactos

Definimos los parámetros de las épocas:

In [52]:
tmin = -0.5  # comienzo de cada época (500ms antes del estímulo)
tmax = 2.0  # final de cada época (2000ms luego del estímulo)

Definimos el periodo de la línea de base (baseline):

In [53]:
baseline = (-0.2, 0)  # 200ms antes del comienzo del estímulo (t = 0)

Ahora elegimos los canales - MEG, EEG y EOG 

In [54]:
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True,
                       stim=False, exclude='bads')

La forma más fácil (¿y quizás también la más peligrosa?) de limpiar los datos es definir parámetros de rechazo de pico-a-pico (rango de amplitud) para gradiómetros, magnetómetros y EOG.

<div class="alert alert-info">
    <b>OBSERVACIÓN</b>:
     <ul>
    <li>El <a href="https://autoreject.github.io/">proyecto de rechacho automático</a> tiene como finalidad resolver este problema. Vea este <a href="https://www.sciencedirect.com/science/article/pii/S1053811917305013">paper</a> para más información.</li>
    </ul>
</div>

In [55]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # esto puede ser altamente dependiente de los datos

Ahora podemos juntar todo esto y crear las épocas:

In [56]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=baseline,
                    reject=reject)

In [ ]:
print(epochs)  # veamos algunos detalles del objeto epochs

Eliminemos explícitamente las épocas que identificamos como _malas_ a través de los umbrales que identificamos anteriormente:

In [ ]:
epochs.drop_bad()  # rechaza las épocas malas mediante reject

In [ ]:
epochs.load_data()  # cargar datos en memoria

## Una mirada más cercana al rechazo de artefactos


Primero, veamos cuáles son los métodos del objeto epochs.
Descomente la línea a continuación y escriba ``epochs.``, se desplegará la lista de métodos disponibles. 

In [60]:
#epochs.

Veamos cómo se eliminaron las épocas.

In [ ]:
%matplotlib inline
epochs.plot_drop_log();

### Puede ser que..., ¿perdimos la mitad de nuestras épocas debido al EOG?

Probablemente podamos hacerlo mejor. Utilicemos la proyección espacial de señales (SSP) basada en PCA para eliminar los patrones espaciales relacionados con el EOG y el ECG.

Este es el flujo de trabajo: primero detectaremos artefactos de EOG y visualizaremos su impacto. Luego calcularemos patrones espaciales para mitigar estos artefactos.

In [ ]:
# Hay una función para crear épocas EOG:
%matplotlib qt
eog_epochs = mne.preprocessing.create_eog_epochs(raw.copy().filter(1, None))
eog_epochs.average().plot_joint();

Veamos dónde aparecen esos segmentos EOG en nuestros datos sin procesar:

In [65]:
%matplotlib qt
raw.plot(events=eog_epochs.events);

Calculemos las proyecciones de SSP basadas en el EOG:

In [66]:
projs_eog, _ = mne.preprocessing.compute_proj_eog(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)

In [ ]:
projs_eog  # veamos cómo son

In [ ]:
%matplotlib inline
mne.viz.plot_projs_topomap(projs_eog, info=epochs.info);

Ahora la pregunta importante es ¿cuántas componentes se deben conservar? Tip: algunas de ellas claramente no parecen patrones de artefactos.

La buena noticia es que no tenemos que decidir __*ahora*__ mismo. Como podemos ver las proyecciones están almacenadas con los datos pero inactivas por el momento. 

Repitamos el procedimiento esta vez para los artefactos de ECG, como son los latidos cardiacos.

In [ ]:
# los mismo para el caso de ECG
%matplotlib qt
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw.copy().filter(1, None))
ecg_epochs.average().plot_joint()

Podemos ver que también nos enfrentamos a contaminación con la señal cardíaca... eso también lo proyectaremos.

In [ ]:
projs_ecg, _ = mne.preprocessing.compute_proj_ecg(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)
mne.viz.plot_projs_topomap(projs_ecg, info=epochs.info);

## Aplicando las proyecciones y visualizando el efecto

Ahora revirtamos nuestro rechazo de artefactos anterior y apliquemos las proyecciones en su lugar. 

In [ ]:
#Eliminamos el EOG de nuestro rechazo aquí:
reject_no_eog = dict(mag=reject['mag'], grad=reject['grad']) 

epochs_clean = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                          picks=picks, baseline=baseline,
                          preload=False,
                          reject=reject_no_eog)

# y luego agregamos las proyecciones de EOG y ECG (¡pero aún no las aplicamos!)
epochs_clean.add_proj(projs_eog + projs_ecg)


Veamos un canal MEG frontal antes de aplicar las proyecciones:

In [72]:
epochs_clean.plot_image(picks='MEG0123', sigma=1.);

¡Ahora apliquemos las proyecciones a una copia y grafiquemos este canal nuevamente!

In [73]:
epochs_proj = epochs_clean.copy().apply_proj()  # aplicamos las proyecciones a una copia

epochs_proj.plot_image(picks='MEG0123', sigma=1.);

Antes establecimos que probablemente no todas las proyecciones capturan parpadeos y artefactos cardíacos. ¡Así que repitamos este procedimiento pero solo proyectemos la _primera_ proyección por tipo de canal!

In [80]:
%matplotlib qt
epochs_clean.del_proj()
epochs_clean.add_proj(projs_eog[::3] + projs_ecg[::3])  # añadimos solo algunas proyecciones SSP
epochs_proj = epochs_clean.copy().apply_proj()  # aplicamos proyecciones a una copia

epochs_proj.plot_image(picks='MEG0123', sigma=1.);

In [ ]:
epochs_proj.info

De esta manera, ahora mantenemos todas las épocas (trials), pero eliminamos los parpadeos y los artefactos cardíacos. Ahora guardaremos los datos con las proyecciones SSP _no aplicadas_.

<div class="alert alert-info">
    <b>OBSERVACIÓN</b>:
     <ul>
    <li>MNE mantiene las proyecciones SSP dentro de la información y permite aplicarlas más tarde.</li>
    </ul>
</div>

#### Algunas reflexiones sobre el rechazo de artefactos

En este ejemplo abordamos los artefactos en este conjunto de datos calculando proyecciones SSP. Sin embargo, hay muchas otras formas de rechazar artefactos:

- marcar artefactos manualmente (inspección visual)
- utilizar umbrales (lo cual falló para estos datos)
- utilizar ICA
- utilizar un pipeline automatizado, por ejemplo el <a href="https://autoreject.github.io/">autoreject project</a>
- etc.

La mejor recomendación es: familiarizate con tus datos (crudos)! 

## Guardando las épocas

In [81]:
# sobreescribamoslas
epochs = epochs_clean

La forma estándar es guardar las épocas como un archivo `.fif` junto con todos los datos del encabezado. Las épocas se guardan con el sufijo `-epo.fif`.

In [ ]:
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')  # creamos el nombre del archivo
epochs_fname

In [ ]:
epochs.save(epochs_fname, overwrite=True) 

## Extra: visualización de  épocas

Vea [ésta página](https://mne.tools/stable/auto_tutorials/epochs/20_visualize_epochs.html) para las opciones de visualización de épocas.

In [83]:
# Ya hemos visto las épocas en un gráfico apilado:

epochs_proj.plot_image(picks='EEG065', sigma=1.);

También podemos ver las épocas en una ventana del navegador de datos:

In [ ]:
%matplotlib qt
epochs.plot();

In [ ]:
epochs.info